## Linear Classifier in TensorFlow 
Using Low Level API in Eager Execution mode

### Load tensorflow

In [0]:
import tensorflow as tf

In [0]:
#Enable Eager Execution if using tensflow version < 2.0
#From tensorflow v2.0 onwards, Eager Execution will be enabled by default


In [180]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### Collect Data

In [0]:
import pandas as pd

In [0]:
df_pricing = pd.read_csv('/content/drive/My Drive/AIML/Residency6th/InternalLab/prices.csv')

### Check all columns in the dataset

In [183]:
df_pricing.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 851264 entries, 0 to 851263
Data columns (total 7 columns):
date      851264 non-null object
symbol    851264 non-null object
open      851264 non-null float64
close     851264 non-null float64
low       851264 non-null float64
high      851264 non-null float64
volume    851264 non-null float64
dtypes: float64(5), object(2)
memory usage: 45.5+ MB


In [184]:
df_pricing.columns

Index(['date', 'symbol', 'open', 'close', 'low', 'high', 'volume'], dtype='object')

### Drop columns `date` and  `symbol`

In [0]:
df_pricing_new = df_pricing.drop(['date','symbol'],axis=1)

In [186]:
df_pricing_new.head()

,open,close,low,high,volume
0,123.430000,125.839996,122.309998,126.250000,2163600.0
1,125.239998,119.980003,119.940002,125.540001,2386400.0
2,116.379997,114.949997,114.930000,119.739998,2489500.0
3,115.480003,116.620003,113.500000,117.440002,2006300.0
4,117.010002,114.970001,114.089996,117.330002,1408600.0


### Consider only first 1000 rows in the dataset for building feature set and target set
Target 'Volume' has very high values. Divide 'Volume' by 1000,000

In [0]:
df_pricing_new['volume'] = df_pricing_new['volume'].apply(lambda x: x/1000000)

In [0]:
pricing = df_pricing_new.head(1000)

### Divide the data into train and test sets

In [0]:
from sklearn.model_selection import train_test_split

X =  pricing.drop("volume", axis=1)
y =  pricing.pop("volume")
Xtrain,Xtest,ytrain,ytest = train_test_split(X, y, test_size=0.30, random_state=1)

#### Convert Training and Test Data to numpy float32 arrays


In [0]:
import numpy as np
Xtrain = np.asarray(Xtrain)
ytrain = np.asarray(ytrain)


### Normalize the data
You can use Normalizer from sklearn.preprocessing

In [0]:
from sklearn import preprocessing
X_normalized = preprocessing.normalize(X, norm='l2')

## Building the Model in tensorflow

1.Define Weights and Bias, use tf.zeros to initialize weights and Bias

In [0]:
#Input features
x = tf.placeholder(shape=[None,4],dtype=tf.float32, name='x-input')

#Normalize the data
x_n = tf.nn.l2_normalize(x,1) 

#Actual Prices
y_ = tf.placeholder(shape=[None],dtype=tf.float32, name='y-input')

W = tf.Variable(tf.zeros(shape=[4,1]), name="Weights")
b = tf.Variable(tf.zeros(shape=[1]),name="Bias")

2.Define a function to calculate prediction

In [0]:
y = tf.add(tf.matmul(x_n,W),b,name='output') #X is normalized, hence X_n

3.Loss (Cost) Function [Mean square error]

In [0]:
loss = tf.reduce_mean(tf.square(y-y_),name='Loss')

4.Function to train the Model

1.   Record all the mathematical steps to calculate Loss
2.   Calculate Gradients of Loss w.r.t weights and bias
3.   Update Weights and Bias based on gradients and learning rate to minimize loss

In [0]:
train_op = tf.train.GradientDescentOptimizer(0.03).minimize(loss)

## Train the model for 100 epochs 
1. Observe the training loss at every iteration
2. Observe Train loss at every 5th iteration

In [0]:
#Lets start graph Execution
sess = tf.Session()

# variables need to be initialized before we can use them
sess.run(tf.global_variables_initializer())

#how many times data need to be shown to model
training_epochs = 100

### Get the shapes and values of W and b

In [197]:
print(W.shape)
print(b.shape)

(4, 1)
(1,)


### Model Prediction on 1st Examples in Test Dataset

In [198]:
for epoch in range(training_epochs):
            
    #Calculate train_op and loss
    _, train_loss = sess.run([train_op,loss],feed_dict={x:Xtrain, y_:ytrain}) # X and Y is like X_Train and Y_Train here.
    
    if epoch % 5 == 0:
        print ('Training loss at step: ', epoch, ' is ', train_loss)

Training loss at step:  0  is  225.87958
Training loss at step:  5  is  203.89143
Training loss at step:  10  is  197.76749
Training loss at step:  15  is  196.0619
Training loss at step:  20  is  195.58687
Training loss at step:  25  is  195.45459
Training loss at step:  30  is  195.41774
Training loss at step:  35  is  195.4075
Training loss at step:  40  is  195.40462
Training loss at step:  45  is  195.40382
Training loss at step:  50  is  195.4036
Training loss at step:  55  is  195.40355
Training loss at step:  60  is  195.40353
Training loss at step:  65  is  195.40355
Training loss at step:  70  is  195.40353
Training loss at step:  75  is  195.40353
Training loss at step:  80  is  195.40353
Training loss at step:  85  is  195.40353
Training loss at step:  90  is  195.40355
Training loss at step:  95  is  195.40353


In [199]:
test_loss = sess.run([loss],feed_dict={x:Xtest, y_:ytest}) 
print ('Training loss at step:is ', test_loss)

Training loss at step:is  [241.01949]


## Classification using tf.Keras

In this exercise, we will build a Deep Neural Network using tf.Keras. We will use Iris Dataset for this exercise.

### Load the given Iris data using pandas (Iris.csv)

In [0]:
iris_data = pd.read_csv('/content/drive/My Drive/AIML/Residency6th/InternalLab/11_Iris.csv')

### Target set has different categories. So, Label encode them. And convert into one-hot vectors using get_dummies in pandas.

In [0]:
keras_data = pd.get_dummies(iris_data,prefix=['Species'], columns=['Species'])

In [202]:
keras_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 8 columns):
Id                         150 non-null int64
SepalLengthCm              150 non-null float64
SepalWidthCm               150 non-null float64
PetalLengthCm              150 non-null float64
PetalWidthCm               150 non-null float64
Species_Iris-setosa        150 non-null uint8
Species_Iris-versicolor    150 non-null uint8
Species_Iris-virginica     150 non-null uint8
dtypes: float64(4), int64(1), uint8(3)
memory usage: 6.4 KB


### Splitting the data into feature set and target set

In [0]:
from sklearn.model_selection import train_test_split

X =  keras_data.drop(['Id',"Species_Iris-setosa",'Species_Iris-versicolor',
       'Species_Iris-virginica'], axis=1)
y =  keras_data[['Species_Iris-setosa', 'Species_Iris-versicolor',
       'Species_Iris-virginica']]
Xtrain,Xtest,ytrain,ytest = train_test_split(X, y, test_size=0.30, random_state=1)

###  Building Model in tf.keras

Build a Linear Classifier model  <br>
1.  Use Dense Layer  with input shape of 4 (according to the feature set) and number of outputs set to 3<br> 
2. Apply Softmax on Dense Layer outputs <br>
3. Use SGD as Optimizer
4. Use categorical_crossentropy as loss function 

In [204]:
from sklearn.neural_network import MLPClassifier
model=MLPClassifier(hidden_layer_sizes=(4))
model.fit(Xtrain,ytrain)

/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
              beta_2=0.999, early_stopping=False, epsilon=1e-08,
              hidden_layer_sizes=4, learning_rate='constant',
              learning_rate_init=0.001, max_iter=200, momentum=0.9,
              n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
              random_state=None, shuffle=True, solver='adam', tol=0.0001,
              validation_fraction=0.1, verbose=False, warm_start=False)

### Model Training 

In [0]:
from keras.models import Sequential
from keras.layers import Dense

model = Sequential()
model.add(Dense(3, input_dim=4, kernel_initializer='normal', activation='softmax'))
#model.add(Dense(3, kernel_initializer='normal'))
# Compile model
model.compile(loss='categorical_crossentropy', optimizer='SGD', metrics= ['accuracy'])

### Model Prediction

In [206]:
model.fit(Xtrain, ytrain, validation_data=(Xtest,ytest), epochs=10, batch_size=5)
# evaluate the model
scores = model.evaluate(X, y)


Train on 105 samples, validate on 45 samples
Epoch 1/10
105/105 [==============================] - 0s 3ms/step - loss: 1.0281 - acc: 0.4000 - val_loss: 1.0060 - val_acc: 0.5778
Epoch 2/10
105/105 [==============================] - 0s 519us/step - loss: 0.9141 - acc: 0.6286 - val_loss: 0.9054 - val_acc: 0.6000
Epoch 3/10
105/105 [==============================] - 0s 507us/step - loss: 0.8262 - acc: 0.7143 - val_loss: 0.7987 - val_acc: 0.6000
Epoch 4/10
105/105 [==============================] - 0s 506us/step - loss: 0.7678 - acc: 0.7238 - val_loss: 0.7653 - val_acc: 0.6000
Epoch 5/10
105/105 [==============================] - 0s 523us/step - loss: 0.7222 - acc: 0.6952 - val_loss: 0.7073 - val_acc: 0.6222
Epoch 6/10
105/105 [==============================] - 0s 534us/step - loss: 0.6728 - acc: 0.7524 - val_loss: 0.7005 - val_acc: 0.6000
Epoch 7/10
105/105 [==============================] - 0s 510us/step - loss: 0.6438 - acc: 0.7810 - val_loss: 0.6360 - val_acc: 0.8444
Epoch 8/10
105/105 

In [207]:
scores[1]*100

69.33333333333334

### Save the Model

In [0]:
model.save("model.dump")

### Build and Train a Deep Neural network with 2 hidden layer  - Optional - For Practice

Does it perform better than Linear Classifier? What could be the reason for difference in performance?